# 주식 데이터를 preprocess 합니다.

In [1]:
import numpy as np
from os.path import join, isfile
from os import listdir
from random import sample
from operator import itemgetter 

In [2]:
root_path = '/data1/stock_data_temp'

# 1. 모든 데이터 로드
# 2. 실시간으로 미니배치를 만들어내는 데이터 로더
# 3. train/test세트를 년도 기준으로

# 1. 모든 데이터를 로드합니다.

In [37]:
"""
1. '날짜': 'date', 2. '종가': 'final_price', 3. '전일비': 'compare_to_prior', 4. '시가': 'start_price',
5. '고가': 'highest_price', 6. '저가': 'lowest_price', 7. '거래량': 'num_of_traded', 8. '회사이름': 'company_name'
9. df_temp = Features.fnMACD(df_temp)
                df_temp = Features.fnBolingerBand(df_temp)
                df_temp = Features.fnRSI(df_temp)
                df_temp = Features.fnStoch(df_temp)
                df_temp = Features.change_prior_to(df_temp)
                df_temp = Features.fnMA(df_temp, m_N=[5, 20, 60, 120, 240])
"""

all_stock_data = list()
for filename in listdir(root_path):
    if not isfile(join(root_path, filename)):
        continue
    print(filename)
    stock_data = np.load(join(root_path, filename))
    all_str_dates = stock_data[:, 0]
    all_str_dates = [int(s.replace('.', '')) for s in all_str_dates]
    stock_data[:, 0] = all_str_dates
    stock_data = stock_data[::-1]
    all_stock_data.append(stock_data)

경방.npy
경보제약.npy
경동도시가스.npy
경동인베스트.npy


In [38]:
all_stock_data[0].shape

(5556, 8)

In [39]:
all_stock_data[0][:, 0]

array([19960625, 19960626, 19960627, ..., 20180725, 20180726, 20180727],
      dtype=object)

# 2. 실시간으로 미니 배치를 만들어내는 데이터 로더를 만듭니다.

## 전체 시퀀스 데이터에서 x에 사용할 데이터와 y에 사용할 데이터를 분리할 수 있는 구조
## 파라미터로 원하는 타겟 x와 y데이터로 변경할 수 있는 함수로 만들어줍니다.

In [40]:
def data_loader(all_stock_data, batch_size, pre_data_length, pro_data_length, process_data):
    num_company = len(all_stock_data)
    num_features = all_stock_data[0].shape[1]
    company_idx = list(range(num_company))
    current_idx = np.array([0 for _ in range(num_company)])
    
    def get_batch(all_stock_data, current_idx, batch_company_idx, pre_data_length, pro_data_length):
        batch_company = [all_stock_data[idx] for idx in batch_company_idx]
        
        # get cur pos of each data in mini batch
        cur_pos = current_idx[batch_company_idx]
        x_batch = np.empty(shape=(0, pre_data_length, num_features))
        y_batch = np.empty(shape=(0, pro_data_length, num_features))
        
        for company, pos in zip(batch_company, cur_pos):
            _x_batch = company[pos: pos+pre_data_length].reshape(1, pre_data_length, num_features)
            _y_batch = company[pos+pre_data_length: pos+pre_data_length+pro_data_length].reshape(1, pro_data_length, num_features)
            
#             print(_new_x_batch)
            x_batch = np.concatenate([x_batch, _x_batch], axis=0)
            y_batch = np.concatenate([y_batch, _y_batch], axis=0)
        
        # update for next pos
        new_pos = [c + 1 for c in cur_pos]
        current_idx[batch_company_idx] = new_pos

        # check if next pos possible, and remove from company_idx if not possible
        for idx, (company, pos) in enumerate(zip(batch_company, new_pos)):
            if company.shape[0] < pos + pre_data_length + pro_data_length:
#                 print(company[0, 7], ' finished at pos:', pos)
                company_idx.remove(batch_company_idx[idx])
                np.delete(current_idx, batch_company_idx[idx])
        
        return x_batch, y_batch
    
    count = 0
    while True:
        while len(company_idx) > 0:
            if count == 0:
                print('New Epoch begins', len(company_idx))
            count += 1
            # random pick from all batch
            if len(company_idx) >= batch_size:
                batch_company_idx = sample(company_idx, batch_size)


                x_batch, y_batch = get_batch(all_stock_data=all_stock_data, current_idx=current_idx, 
                                                     pre_data_length=pre_data_length, pro_data_length=pro_data_length,
                                                     batch_company_idx=batch_company_idx)

            else: # if number of company less than batch, then company should go for parallel
                batch_company_idx = company_idx[:]
                x_batch = list()
                y_batch = list()
                idx = 0
                while len(x_batch) < batch_size and len(company_idx) > 0:
                    if idx >= len(company_idx):
                        idx = 0
                    batch_company_idx = company_idx[idx]
                    idx += 1
                    idx = idx % len(company_idx)
                    _x_batch, _y_batch = get_batch(all_stock_data=all_stock_data, current_idx=current_idx, 
                                                         pre_data_length=pre_data_length, pro_data_length=pro_data_length,
                                                         batch_company_idx=[batch_company_idx])
                    x_batch.append(_x_batch)
                    y_batch.append(_y_batch)

                x_batch = np.concatenate(x_batch, axis=0)
                y_batch = np.concatenate(y_batch, axis=0)

            # 마지막 처리를 해줍니다.
            x_batch, y_batch = process_data(x_batch, y_batch)

            yield x_batch, y_batch
        
        company_idx = list(range(num_company))
        current_idx = np.array([0 for _ in range(num_company)])

# 단, 케라스 fit_generator에서 사용할 때에는 while True 루프가 필요합니다. 따라서 아래와 같이 고칩니다.

In [ ]:
def data_loader(all_stock_data, batch_size, pre_data_length, pro_data_length, process_data):
    num_company = len(all_stock_data)
    num_features = all_stock_data[0].shape[1]
    company_idx = list(range(num_company))
    current_idx = np.array([0 for _ in range(num_company)])
    
    def get_batch(all_stock_data, current_idx, batch_company_idx, pre_data_length, pro_data_length):
        batch_company = [all_stock_data[idx] for idx in batch_company_idx]
        
        # get cur pos of each data in mini batch
        cur_pos = current_idx[batch_company_idx]
        x_batch = np.empty(shape=(0, pre_data_length, num_features))
        y_batch = np.empty(shape=(0, pro_data_length, num_features))
        
        for company, pos in zip(batch_company, cur_pos):
            _x_batch = company[pos: pos+pre_data_length].reshape(1, pre_data_length, num_features)
            _y_batch = company[pos+pre_data_length: pos+pre_data_length+pro_data_length].reshape(1, pro_data_length, num_features)
            
#             print(_new_x_batch)
            x_batch = np.concatenate([x_batch, _x_batch], axis=0)
            y_batch = np.concatenate([y_batch, _y_batch], axis=0)
        
        # update for next pos
        new_pos = [c + 1 for c in cur_pos]
        current_idx[batch_company_idx] = new_pos

        # check if next pos possible, and remove from company_idx if not possible
        for idx, (company, pos) in enumerate(zip(batch_company, new_pos)):
            if company.shape[0] < pos + pre_data_length + pro_data_length:
#                 print(company[0, 7], ' finished at pos:', pos)
                company_idx.remove(batch_company_idx[idx])
                np.delete(current_idx, batch_company_idx[idx])
        
        return x_batch, y_batch
    
    count = 0
    while True:
        while len(company_idx) > 0:
            if count == 0:
                print('New Epoch begins', len(company_idx))
            count += 1
            # random pick from all batch
            if len(company_idx) >= batch_size:
                batch_company_idx = sample(company_idx, batch_size)


                x_batch, y_batch = get_batch(all_stock_data=all_stock_data, current_idx=current_idx, 
                                                     pre_data_length=pre_data_length, pro_data_length=pro_data_length,
                                                     batch_company_idx=batch_company_idx)

            else: # if number of company less than batch, then company should go for parallel
                batch_company_idx = company_idx[:]
                x_batch = list()
                y_batch = list()
                idx = 0
                while len(x_batch) < batch_size and len(company_idx) > 0:
                    if idx >= len(company_idx):
                        idx = 0
                    batch_company_idx = company_idx[idx]
                    idx += 1
                    idx = idx % len(company_idx)
                    _x_batch, _y_batch = get_batch(all_stock_data=all_stock_data, current_idx=current_idx, 
                                                         pre_data_length=pre_data_length, pro_data_length=pro_data_length,
                                                         batch_company_idx=[batch_company_idx])
                    x_batch.append(_x_batch)
                    y_batch.append(_y_batch)

                x_batch = np.concatenate(x_batch, axis=0)
                y_batch = np.concatenate(y_batch, axis=0)

            # 마지막 처리를 해줍니다.
            x_batch, y_batch = process_data(x_batch, y_batch)

            yield x_batch, y_batch
        
        company_idx = list(range(num_company))
        current_idx = np.array([0 for _ in range(num_company)])

# 2. 케라스에서는 custom batch loader를 사용시(fit_generator) estimated step이 필요합니다. 따라서 미리 몇 스텝인지 계산해야 합니다.

In [41]:
# calculating total estimated steps
def get_estimated_steps(all_stock_data, pre_data_length, pro_data_length, batch_size):
    
    all_length = pre_data_length + pro_data_length

    total_steps = 0
    for stock_data in all_stock_data:
        total_steps += (stock_data.shape[0] - all_length + 1)
    #     print(total_steps)

    return int(np.ceil(total_steps / batch_size))

## 테스트 해봅니다. 실제 데이터 로더의 스텝은 idx + 1로 계산하고, get_estimated_steps로 예상 스텝과 비교합니다.

In [42]:
batch_size = 4
pre_data_length = 20
pro_data_length = 1

def process_data(x_batch, y_batch):
    return x_batch, y_batch

loader = data_loader(all_stock_data=all_stock_data, batch_size=batch_size, 
                     pre_data_length=pre_data_length, pro_data_length=pro_data_length, 
                     process_data=process_data)
for idx, (x_batch, y_batch) in enumerate(loader):
    continue
    
estimated_steps = get_estimated_steps(all_stock_data=all_stock_data, 
                                              pre_data_length=pre_data_length, pro_data_length=pro_data_length, 
                                              batch_size=batch_size)
    
print('Number of steps:', idx + 1)
print('Estimated Steps:', estimated_steps)

Number of steps: 2967
Estimated Steps: 2967


## 데이터도 확인해보구요.

In [46]:
batch_size = 4
pre_data_length = 20
pro_data_length = 1

loader = data_loader(all_stock_data=all_stock_data, batch_size=batch_size, 
                     pre_data_length=pre_data_length, pro_data_length=pro_data_length, 
                     process_data=process_data)
for idx, (x_batch, y_batch) in enumerate(loader):
    break
    
print('첫 번째 배치')
print('x_batch.shape:', x_batch.shape)
print('x_batch:', x_batch)
print('y_batch.shape:', y_batch.shape)
print('y_batch', y_batch)

print('')
print('')
for idx, (x_batch, y_batch) in enumerate(loader):
    continue
    
print('마지막 배치')
print('x_batch.shape:', x_batch.shape)
print('x_batch:', x_batch)
print('y_batch.shape:', y_batch.shape)
print('y_batch', y_batch)

첫 번째 배치
x_batch.shape: (4, 20, 8)
x_batch: [[[19970307 43200.0 43200.0 43200.0 43200.0 43200.0 10.0 '경동인베스트']
  [19970308 46600.0 3400.0 46600.0 46600.0 46600.0 25020.0 '경동인베스트']
  [19970310 47500.0 900.0 50300.0 50300.0 47500.0 55720.0 '경동인베스트']
  [19970311 44600.0 2900.0 47000.0 47000.0 44100.0 108960.0 '경동인베스트']
  [19970312 43600.0 1000.0 45200.0 45200.0 43500.0 105650.0 '경동인베스트']
  [19970313 42100.0 1500.0 43600.0 44100.0 42100.0 13890.0 '경동인베스트']
  [19970314 41000.0 1100.0 41600.0 42000.0 40200.0 11050.0 '경동인베스트']
  [19970315 41000.0 0.0 40500.0 41500.0 40500.0 6470.0 '경동인베스트']
  [19970317 43000.0 2000.0 40500.0 43000.0 40500.0 9650.0 '경동인베스트']
  [19970318 43500.0 500.0 43000.0 44000.0 43000.0 11710.0 '경동인베스트']
  [19970319 43500.0 0.0 44000.0 44000.0 41500.0 5710.0 '경동인베스트']
  [19970320 42300.0 1200.0 43000.0 43000.0 42000.0 12570.0 '경동인베스트']
  [19970321 41300.0 1000.0 41500.0 41800.0 41100.0 3860.0 '경동인베스트']
  [19970322 42000.0 700.0 41000.0 42000.0 41000.0 1740.0 '경동인베스트']
  [19

## 기본은 갖추어졌네요. 그럼 이번에는 y_batch를 regression용 금액으로 바꾸고, y_batch를 classification용 금액으로도 바꾸어 보겠습니다.

## t+1 금액을 예측하기 위한 Regression 데이터 예제입니다.

In [45]:
batch_size = 4
pre_data_length = 20
pro_data_length = 1

# regression example
def process_regression_data(x_batch, y_batch):
    
    y_batch = y_batch[:, :, 1] # final price index = 1
    
    return x_batch, y_batch

loader = data_loader(all_stock_data=all_stock_data, batch_size=batch_size, 
                     pre_data_length=pre_data_length, pro_data_length=pro_data_length, 
                     process_data=process_regression_data)
for idx, (x_batch, y_batch) in enumerate(loader):
    break
    
print('첫 번째 배치')
print('x_batch.shape:', x_batch.shape)
print('x_batch:', x_batch)
print('y_batch.shape:', y_batch.shape)
print('y_batch', y_batch)

첫 번째 배치
x_batch.shape: (4, 20, 8)
x_batch: [[[19960625 42000.0 0.0 42000.0 42000.0 42000.0 0.0 '경방']
  [19960626 42000.0 0.0 42000.0 42000.0 42000.0 0.0 '경방']
  [19960627 40500.0 1500.0 40500.0 40500.0 40500.0 120.0 '경방']
  [19960628 40500.0 0.0 40500.0 40500.0 40500.0 10.0 '경방']
  [19960629 40500.0 0.0 40500.0 40500.0 40500.0 0.0 '경방']
  [19960701 40000.0 500.0 40000.0 40000.0 40000.0 50.0 '경방']
  [19960702 40000.0 0.0 40000.0 40000.0 40000.0 0.0 '경방']
  [19960703 40000.0 0.0 40000.0 40000.0 40000.0 50.0 '경방']
  [19960704 39500.0 500.0 39500.0 39500.0 39500.0 110.0 '경방']
  [19960705 39500.0 0.0 39500.0 39500.0 39500.0 0.0 '경방']
  [19960706 39500.0 0.0 39500.0 39500.0 39500.0 0.0 '경방']
  [19960708 39000.0 500.0 39000.0 39000.0 39000.0 130.0 '경방']
  [19960709 39000.0 0.0 39000.0 39000.0 39000.0 100.0 '경방']
  [19960710 40000.0 1000.0 40000.0 40000.0 40000.0 10.0 '경방']
  [19960711 40500.0 500.0 40500.0 40500.0 40500.0 0.0 '경방']
  [19960712 40500.0 0.0 40500.0 40500.0 40500.0 0.0 '경방']
  [

## t+1이 전일에 비해 상승이면 0 하락 또는 보합이면 1로 바꾼 classification 데이터셋 예제입니다.

In [44]:
batch_size = 4
pre_data_length = 20
pro_data_length = 1

# classificaiton example
def process_classification_data(x_batch, y_batch):
    
    y_batch = y_batch[:, :, 2] # final price index = 1
    y_batch[np.where(y_batch <= 0)[0], 0] = 0
    y_batch[np.where(y_batch > 0)[0], 0] = 1
    
    return x_batch, y_batch

loader = data_loader(all_stock_data=all_stock_data, batch_size=batch_size, 
                     pre_data_length=pre_data_length, pro_data_length=pro_data_length, 
                     process_data=process_classification_data)
for idx, (x_batch, y_batch) in enumerate(loader):
    break
    
print('첫 번째 배치')
print('x_batch.shape:', x_batch.shape)
print('x_batch:', x_batch)
print('y_batch.shape:', y_batch.shape)
print('y_batch', y_batch)

첫 번째 배치
x_batch.shape: (4, 20, 8)
x_batch: [[[19970307 43200.0 43200.0 43200.0 43200.0 43200.0 10.0 '경동인베스트']
  [19970308 46600.0 3400.0 46600.0 46600.0 46600.0 25020.0 '경동인베스트']
  [19970310 47500.0 900.0 50300.0 50300.0 47500.0 55720.0 '경동인베스트']
  [19970311 44600.0 2900.0 47000.0 47000.0 44100.0 108960.0 '경동인베스트']
  [19970312 43600.0 1000.0 45200.0 45200.0 43500.0 105650.0 '경동인베스트']
  [19970313 42100.0 1500.0 43600.0 44100.0 42100.0 13890.0 '경동인베스트']
  [19970314 41000.0 1100.0 41600.0 42000.0 40200.0 11050.0 '경동인베스트']
  [19970315 41000.0 0.0 40500.0 41500.0 40500.0 6470.0 '경동인베스트']
  [19970317 43000.0 2000.0 40500.0 43000.0 40500.0 9650.0 '경동인베스트']
  [19970318 43500.0 500.0 43000.0 44000.0 43000.0 11710.0 '경동인베스트']
  [19970319 43500.0 0.0 44000.0 44000.0 41500.0 5710.0 '경동인베스트']
  [19970320 42300.0 1200.0 43000.0 43000.0 42000.0 12570.0 '경동인베스트']
  [19970321 41300.0 1000.0 41500.0 41800.0 41100.0 3860.0 '경동인베스트']
  [19970322 42000.0 700.0 41000.0 42000.0 41000.0 1740.0 '경동인베스트']
  [19

# 3. Train/Test 세트 분리 로직

In [52]:
def split_train_test_by_year(all_stock_data, year): # year 이전은 train, year 해당 년 및 이후는 test
    train_stock_data = list()
    test_stock_data = list()
    for stock_data in all_stock_data:
        indices = np.where(stock_data[:, 0] >= (year * 10000))[0]
        test_stock_data.append(stock_data[indices])
        
        indices = np.where(stock_data[:, 0] < (year * 10000))[0]
        train_stock_data.append(stock_data[indices])
    
    return train_stock_data, test_stock_data
    

In [50]:
np.where(all_stock_data[0][:, 0] > 20170000)

(array([5172, 5173, 5174, 5175, 5176, 5177, 5178, 5179, 5180, 5181, 5182,
        5183, 5184, 5185, 5186, 5187, 5188, 5189, 5190, 5191, 5192, 5193,
        5194, 5195, 5196, 5197, 5198, 5199, 5200, 5201, 5202, 5203, 5204,
        5205, 5206, 5207, 5208, 5209, 5210, 5211, 5212, 5213, 5214, 5215,
        5216, 5217, 5218, 5219, 5220, 5221, 5222, 5223, 5224, 5225, 5226,
        5227, 5228, 5229, 5230, 5231, 5232, 5233, 5234, 5235, 5236, 5237,
        5238, 5239, 5240, 5241, 5242, 5243, 5244, 5245, 5246, 5247, 5248,
        5249, 5250, 5251, 5252, 5253, 5254, 5255, 5256, 5257, 5258, 5259,
        5260, 5261, 5262, 5263, 5264, 5265, 5266, 5267, 5268, 5269, 5270,
        5271, 5272, 5273, 5274, 5275, 5276, 5277, 5278, 5279, 5280, 5281,
        5282, 5283, 5284, 5285, 5286, 5287, 5288, 5289, 5290, 5291, 5292,
        5293, 5294, 5295, 5296, 5297, 5298, 5299, 5300, 5301, 5302, 5303,
        5304, 5305, 5306, 5307, 5308, 5309, 5310, 5311, 5312, 5313, 5314,
        5315, 5316, 5317, 5318, 5319, 

In [54]:
train_stock_data, test_stock_data = split_train_test_by_year(all_stock_data, year=2017)

In [56]:
train_stock_data

[array([[19960625, 42000.0, 0.0, ..., 42000.0, 0.0, '경방'],
        [19960626, 42000.0, 0.0, ..., 42000.0, 0.0, '경방'],
        [19960627, 40500.0, 1500.0, ..., 40500.0, 120.0, '경방'],
        ...,
        [20161227, 15900.0, 250.0, ..., 15600.0, 66299.0, '경방'],
        [20161228, 15550.0, 350.0, ..., 15550.0, 44405.0, '경방'],
        [20161229, 15300.0, 250.0, ..., 15300.0, 34004.0, '경방']],
       dtype=object),
 array([[20150629, 34250.0, 4250.0, ..., 30000.0, 5743250.0, '경보제약'],
        [20150630, 32000.0, 2250.0, ..., 28100.0, 1511196.0, '경보제약'],
        [20150701, 33300.0, 1300.0, ..., 32600.0, 831224.0, '경보제약'],
        ...,
        [20161227, 13750.0, 0.0, ..., 13550.0, 68318.0, '경보제약'],
        [20161228, 14050.0, 300.0, ..., 13550.0, 49145.0, '경보제약'],
        [20161229, 13950.0, 100.0, ..., 13800.0, 22465.0, '경보제약']],
       dtype=object),
 array([], shape=(0, 8), dtype=object),
 array([[19970307, 43200.0, 43200.0, ..., 43200.0, 10.0, '경동인베스트'],
        [19970308, 46600.0, 3400.0,

In [55]:
test_stock_data

[array([[20170102, 14800.0, 500.0, ..., 14650.0, 89378.0, '경방'],
        [20170103, 14900.0, 100.0, ..., 14750.0, 23480.0, '경방'],
        [20170104, 14800.0, 100.0, ..., 14700.0, 28288.0, '경방'],
        ...,
        [20180725, 12650.0, 400.0, ..., 12550.0, 45778.0, '경방'],
        [20180726, 12650.0, 0.0, ..., 12400.0, 23579.0, '경방'],
        [20180727, 12900.0, 250.0, ..., 12550.0, 19363.0, '경방']],
       dtype=object),
 array([[20170102, 13600.0, 350.0, ..., 13550.0, 38914.0, '경보제약'],
        [20170103, 13900.0, 300.0, ..., 13450.0, 44309.0, '경보제약'],
        [20170104, 14050.0, 150.0, ..., 13900.0, 59080.0, '경보제약'],
        ...,
        [20180725, 11750.0, 200.0, ..., 11650.0, 55984.0, '경보제약'],
        [20180726, 12000.0, 250.0, ..., 11750.0, 18309.0, '경보제약'],
        [20180727, 12150.0, 150.0, ..., 12000.0, 14377.0, '경보제약']],
       dtype=object),
 array([[20170512, 43700.0, 8400.0, ..., 41100.0, 506129.0, '경동도시가스'],
        [20170515, 48000.0, 4300.0, ..., 44150.0, 608556.0, '경동도시가스

# 여기까지 했으니, 이제 주가 데이터를 분석해 보도록 할까요?